In [ ]:
import argparse
import multiprocessing
import os
import time
from collections import defaultdict
from itertools import chain, islice
from math import ceil

import numpy as np
import yaml

from config import SPECS_COMPLETE_YAMLS, HEURISTIC_RESULTS_DIR
from model import DC, LPAC
from util import in_notebook

In [ ]:
# user inputs
if in_notebook():
    f_min = 0
    f_max = 20
    f_inc = 1
    r_hat = 3
    pftype = 'dc'
    casestudy = 'imelda'
    approach = 'stochastic'
    omega = 1
    num_chunks = 32
    num_processes = 8
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--fmin')
    parser.add_argument('--fmax')
    parser.add_argument('--finc')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype')
    parser.add_argument('--approach')
    parser.add_argument('--omega')
    parser.add_argument('--numchunks')
    parser.add_argument('--numprocesses')
    args = parser.parse_args()
    f_min = int(args.fmin)
    f_max = int(args.fmax)
    f_inc = int(args.finc)
    r_hat = int(args.rhat)
    pftype = str(args.pftype)
    casestudy = str(args.casestudy)
    approach = str(args.approach)
    omega = int(args.omega)
    num_chunks = int(args.numchunks)
    num_processes = int(args.numprocesses)

In [ ]:
def heuristic_perf(solutions, modelcls, omega, specs_main, specs_scenario, num_threads):
    MODEL = modelcls(**specs_main, **specs_scenario)

    # initialize the instance with the "do nothing" first-stage solution
    for (k, r) in MODEL.x:
        MODEL.x[k, r].lb = 0
        MODEL.x[k, r].ub = 0
    # set flags
    MODEL.model.setParam('OutputFlag', 0)
    MODEL.model.setParam('Threads', num_threads)
    MODEL.model.setParam('Presolve', 2)
    MODEL.model.setParam('Method', 2)
    
    perf = dict()
    # evaluate heuristic solutions
    for key, solution in solutions.items():
        print((*key, omega))
        t0 = time.time()
        f, weight_load, weight_flow = key
        immutable_solution = tuple(sorted(chain.from_iterable(solution)))
        for (k, r) in immutable_solution:
            MODEL.x[k, r].lb = 1
            MODEL.x[k, r].ub = 1
        MODEL.con_resource_hi.RHS = f
        MODEL.update()
        MODEL.solve()
        perf[(int(f), float(weight_load), float(weight_flow), omega)] = MODEL.model.getObjective().getValue()
        for (k, r) in immutable_solution:
            MODEL.x[k, r].lb = 0
            MODEL.x[k, r].ub = 0
        tf = time.time()
    return perf

def chunks(data, size):
    it = iter(data)
    for i in range(0, len(data), size):
        yield {k:data[k] for k in islice(it, size)}

In [ ]:
# load grid data
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype]) as fh:
    specs_main = yaml.load(fh, Loader=yaml.Loader)
    specs_main['options']['approach'] = approach
    for key, val in specs_main['r_hat'].items():
        specs_main['r_hat'][key] = min(val, r_hat)
    for key in list(specs_main['xi']):
        (k, r, _) = key
        if r > r_hat:
            specs_main['xi'].pop(key)
    for k in specs_main['R']:
        specs_main['R'][k] = [i for i in range(1, min(max(specs_main['R'][k]), r_hat) + 1)]
    Omega = specs_main.pop('Omega')
    probability = specs_main.pop('probability')
    xi = specs_main.pop('xi')
    specs_scenario = dict()
    specs_scenario['Omega'] = [omega]
    specs_scenario['probability'] = {omega: 1.0}
    specs_scenario['xi'] = {(k, r, omegap): 1.0
                            for (k, r, omegap) in xi
                            if omega == omegap}

# load heuristic solution data
heur_sols_filename = os.path.join(HEURISTIC_RESULTS_DIR, approach, f'solutions-{casestudy}-r{r_hat}.yaml')
with open(heur_sols_filename) as fh:
    solutions = yaml.load(fh, Loader=yaml.Loader)

model_classes = {
    'dc': DC,
    'lpacc': LPAC,
    'lpacf': LPAC,
    'qpac': LPAC
}

model_cls = model_classes[pftype]

In [ ]:
num_threads = int(np.floor(multiprocessing.cpu_count() / num_processes))
size_chunks = ceil(len(solutions) / num_chunks)

args = [(chunk, model_cls, omega, specs_main, specs_scenario, num_threads)
        for chunk in chunks(solutions, size_chunks)]

with multiprocessing.Pool(num_processes) as pool:
    results = pool.starmap(heuristic_perf, args)

In [ ]:
perf = dict()
for result in results:
    perf.update(result)

heur_perf_filename = os.path.join(HEURISTIC_RESULTS_DIR, approach, f'perf-{pftype}-{casestudy}-{omega}-r{r_hat}.yaml')
with open(heur_perf_filename, 'w') as fh:
    yaml.dump(perf, fh)